<a href="https://colab.research.google.com/github/AmanBansall/IntelligentExcuseGenerator/blob/main/updated_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from transformers import pipeline
import os, getpass

!jupyter nbconvert --ClearMetadataPreprocessor.enabled=True --to notebook --inplace your_notebook.ipynb

!pip install transformers requests -q

# Securely input Hugging Face token
os.environ["HF_TOKEN"] = getpass.getpass("Enter your Hugging Face token (hidden): ")

# Load AI Model,here,deepseek
generator = pipeline("text-generation", model="deepseek-ai/DeepSeek-R1-Distill-Llama-8B",
                     token=os.environ["HF_TOKEN"])

# Load CSV file
url = "/content/drive/MyDrive/Colab Notebooks/excuses_updated.csv"
df = pd.read_csv(url)



def generate_ai_excuse(scenario, urgency):
   prompt = f"Give me a creative excuse for {scenario} with {urgency} urgency."
   ai_excuse = generator(prompt, max_length=20, num_return_sequences=1)[0]["generated_text"]
   return ai_excuse.strip()

# Function to get CSV excuses
def get_csv_excuses(scenario, urgency):
    filtered = df[(df['Scenario'] == scenario) & (df['Urgency'] == urgency)]
    return filtered['Excuse'].tolist()


# Function to add a new excuse
def add_excuse(scenario, urgency, excuse, path=url):
    local_df = pd.read_csv(path)
    new_row = pd.DataFrame([[scenario, urgency, excuse]],
                           columns=["Scenario", "Urgency", "Excuse"])
    local_df = pd.concat([local_df, new_row], ignore_index=True)
    local_df.to_csv(path, index=False)
    print(f"Saved successfully! Total rows: {len(local_df)}")

# Interaction with user
scenario = input("Enter scenario (Work, School, Social, Family): ").strip().title()
urgency = input("Enter urgency (High, Medium, Low): ").strip().title()

# AI Generated excuse
print("\n AI-generated excuse:")
ai_excuse = generate_ai_excuse(scenario, urgency)
print(ai_excuse)

# Ask if user likes it
choice = input("\nDo you like this excuse? (yes/no): ").strip().lower()
if choice == "no":
    csv_excuses = get_csv_excuses(scenario, urgency)
    if csv_excuses:
        print("\n Excuses from CSV:")
        for i, excuse in enumerate(csv_excuses, 1):
            print(f"{i}. {excuse}")
    else:
        print(" No CSV excuses found for this combination.")

# Option to add new excuse
choice2 = input("\nDo you want to add a new excuse? (yes/no): ").strip().lower()
if choice2 == "yes":
    scenario_input = input("Enter the scenario (e.g., Work, School, Meeting): ").strip().title()
    urgency_input = input("Enter the urgency level (e.g., High, Low, Medium): ").strip().title()
    excuse_input = input("Enter your excuse: ").strip()
    add_excuse(scenario_input, urgency_input, excuse_input)
    print("\nUpdated data preview:")
    print(pd.read_csv(url).tail())

KeyboardInterrupt: 